환경설정

In [76]:
import os
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import tqdm
import jpype
import ast

In [77]:
folder_path = '/Users/jaesolshin/Documents/GitHub/youtube_dashboard'
file_path = os.path.join(folder_path, 'KPOP_comments_merged_preprocessed_with_nouns.csv')
comments_df = pd.read_csv(file_path)

In [78]:
sample_df = comments_df.copy()
sample_df['word_list'] = sample_df['word_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [79]:
sample_df.head()

,Unnamed: 0.1,Group,Title,Unnamed: 0,comment_type,comment_id_key,parent_id_key,comment,author,date,likes,word_list
0,0,NMIXX,별별별 (See that),21,node,UgyiK-QaarUUumBCjUJ4AaABAg,NaN,엔믹스 노래 중독성 있고 좋은데 왜 안뜨는지 진심 의문..,@lumi1216,2024-10-01T13:40:14Z,4,"[중독, 진심, 노래, 믹스, 의문]"
1,1,NMIXX,별별별 (See that),32,node,Ugw_6c4z7iSqIjzfgZB4AaABAg,NaN,대쉬랑 비슷한것 같기도하고 이젠 좀 다른 대중성있는 노래로갔으면하는 아쉬움이있네요 ...,@써니노,2024-10-01T09:56:20Z,1,"[기도, 금도, 대쉬, 아쉬움, 노래, 대중성]"
2,2,NMIXX,별별별 (See that),43,node,UgxLdDuk-Ji3RKARe_h4AaABAg,NaN,4본부장아 피씨로 엘범 클릭하면 별별별 뮤비로 연결 안되고 수정별인가 그 장면 있는...,@고해진-i4g,2024-10-01T05:16:15Z,4,"[간다, 엘범, 피씨, 천만, 뮤비, 뷰수, 관리, 재생, 바로, 클릭, 별별, 노..."
3,3,NMIXX,별별별 (See that),69,node,Ugz-Tnp-d-MLAQSXvQF4AaABAg,NaN,이야 노래 종나 멋있네,@박종한-i9u,2024-09-30T18:33:22Z,10,[노래]
4,4,NMIXX,별별별 (See that),71,node,UgwlfRfRmVnAsHIREut4AaABAg,NaN,외모췍을 필두로 더더더더 빵빵뜨길...리더로서 해원이 얼마나 노력했을까...멤버들 ...,@egi3581,2024-09-30T17:22:58Z,6,"[로서, 더더, 해원, 세대, 리더, 노력, 묵자, 빵빵, 얼마나, 걸그룹, 멤버,..."


In [80]:
# 새롭게 발견한 불용어 제거
stop_words = [
    "여기", "이제", "마지막", "때문", "이유", "이후", "이전", "모든", "위해", "이름", 
    "시작", "의미", "완전", "관련", "무슨", "사실", "절대", "누가", "소리", "한번",
    "어디", "원래", "바로", "자신", "누구", "얘기", "가지", "건가", "수가", "빠다"
    "거기", "수도", "너희", "이야기", "조금", "이후", "해도", "그게", "일단", "그날",
    "그거", "타고", "매우", "라인", "일이", "퍼트", "거의", "부터", "그대로", "보임",
    "거임", "이기", "보기", "차라리", "처럼", "그게", "하자", "웨이", "이기", "건가",
    "간다", "게다가", "저번", "바로", "로서", "고간", "수가", "자체"
]

# word_list 열에서 stopwords에 포함된 단어들을 제거하는 작업
sample_df['word_list'] = sample_df['word_list'].apply(lambda word_list: [word for word in word_list if word not in stop_words])

In [81]:
correction_dict = {
    "하나": "하나님",
    "아이": "아이브",
    "믹스": "엔믹스",
    "프티": "피프티",
    "스테이": "스테이씨",
    "보이": "하입보이",
    "버니": "버니즈",
    "뉴진": "뉴진스",
    "수록": "수록곡",
    "비주": "비주얼",
    "그네": "마그네틱",
    "전체": "전체적",
    "대한": "대한민국",
    "침착": "침착맨",
    "아무": "아무리, 아무나",
    "오엠": "오엠지",
    "최고다": "최고",
    "별별": "별별별",
    "너비": "워너비",
    "진영": "박진영",
    "투바": "투바투",
    "홍준": "전홍준",
    "가즈": "가즈아",
    "둥이": "핍둥이",
    "현이": "채현",
    "아이즈": "아이즈원",
    "디오": "아센디오",
    "아센": "아센디오",
    "러브": "러브다이브",
    "국문": "중국문화, 한국문화",
    "대혁명": "문화대혁명",
    "애프터": "애프터라이크",
    "순이": "콩순이",
    "즈하": "카즈하",
    "하이": "하이브",
    "병기": "정병기",
    "스파": "에스파",
    "파가": "에스파",
    "레벨": "넥스트 레벨, 레드벨벳",
    "넥스트": "넥스트 레벨",
    "파의": "에스파",
    "레드": "레드벨벳",
    "벨벳": "레드벨벳",
    "에이": "에이셉",
    "지원이": "박지원",
    "엠피": "에스엠피",
    "오버": "오버워치",
    "피어": "피어나",
    "가람이": "김가람",
    "파의": "에스파",
    "스키": "스키즈",
    "최고다": "최고",
    "힙해": "힙한",
    "힙하": "힙한"
}

In [82]:
# word_list 열에서 correction_dict의 키를 찾아서 대체 및 제거하는 함수 정의
def replace_or_remove_keywords(row):
    word_list = row['word_list']
    comment = row['comment']

    # 새로운 word_list를 저장할 리스트
    new_word_list = []

    # word_list에서 correction_dict의 키를 확인하고 대체 또는 제거
    for word in word_list:
        if word in correction_dict:  # word가 correction_dict의 키인지 확인
            possible_replacements = correction_dict[word]
            
            # replacement가 여러 개일 경우 분할
            if ',' in possible_replacements:
                possible_replacements = possible_replacements.split(', ')
            else:
                possible_replacements = [possible_replacements]
            
            # comment에서 각 replacement가 있는지 확인
            replaced = False
            for replacement in possible_replacements:
                if replacement in comment:
                    new_word_list.append(replacement)  # 대체할 값이 있으면 추가
                    replaced = True
                    break  # 대체가 이루어지면 반복 중단
            
            # 대체가 이루어지지 않으면 word를 제거 (추가하지 않음)
        else:
            # word가 correction_dict에 없으면 유지
            new_word_list.append(word)

    return new_word_list


# word_list 열을 수정하는 apply 함수
sample_df['word_list'] = sample_df.apply(replace_or_remove_keywords, axis=1)

In [83]:
# 특정 단어가 포함된 댓글 조회하기

def search_from_comment(hot_word, dataframe=sample_df, likes=False, n_comments=10, random_state=42):

    # 데이터 프레임에서 hot_word를 포함하고 있는 행 필터링
    filtered_df = dataframe[dataframe['comment'].str.contains(hot_word, na=False)]
    print(len(filtered_df))

    # 좋아요 기준 정렬
    if likes:
        filtered_df = filtered_df[['comment','word_list','likes']].sort_values(by='likes', ascending=False)
        result = filtered_df[:n_comments]
    
    # 랜덤으로 조회
    else :
        result = filtered_df[['comment','word_list','likes']].sample(n = n_comments, random_state=42)
    
    return result


# 특정 단어가 포함된 댓글 조회하기
def search_from_word_list(hot_word, dataframe=sample_df, likes=False, n_comments=10, random_state=42):

    # 데이터 프레임에서 hot_word를 포함하고 있는 행 필터링
    filtered_df = dataframe[dataframe['word_list'].apply(lambda x: hot_word in x)]
    print(f"필터링된 댓글 수: {len(filtered_df)}")

    # 필터링된 결과가 없을 경우 빈 데이터프레임 반환
    if len(filtered_df) == 0:
        return pd.DataFrame(columns=['comment', 'word_list', 'likes'])
    
    # 좋아요 기준 정렬
    if likes:
        filtered_df = filtered_df[['comment', 'word_list', 'likes']].sort_values(by='likes', ascending=False)
        result = filtered_df[:n_comments]
    
    # 랜덤으로 조회
    else:
        result = filtered_df[['comment', 'word_list', 'likes']].sample(n=n_comments, random_state=random_state)
    
    return result

In [84]:
search_from_word_list('아이브', likes=True)

필터링된 댓글 수: 27121


,comment,word_list,likes
587608,아이브의 케이팝 분위기는 정말 대단해요. 매번 우리를 놀라게 해요.,"[분위기, 케이팝, 아이브, 매번]",7492
610875,한창 해외판으로 나가는 돌판인데 아이브는 역시 아이브구나.. 한국풍으로 기강 딱 잡겠네,"[한창, 해외, 기강, 돌판, 구나, 아이브, 한국]",4385
581639,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 진짜 초등학생들 취향저격일듯.. 나 초딩이엇으면 아이브놀이 햇...,"[참고, 초등학생, 성인, 저격, 봣슴니, 재밋, 취향, 초딩, 엇으, 아이브]",3672
187932,아이브 꽃길만 걷자 !! 절대 지켜!!,"[꽃길, 아이브]",3316
613545,우와.. 아니 어떻게 아이브 멤버들은 다 매달 컴백마다 리즈를 찍는거지,"[리즈, 컴백, 멤버, 아이브, 매달]",3094
580157,내가보기엔 맨처음에 물속에서 가을이를 부르던 아이브가 흑이브임. 유독 스산하게 연출...,"[스산하, 주술, 장면, 해리포터, 정신, 거울, 연출, 과정, 뮤비, 아센디오, ...",2712
607776,자기복제 없이 매번 새로운 색깔로 도전하는 거 진짜 호감 ㅋㅋ 그렇다고 아이브만의 ...,"[호감, 정체, 색깔, 매번, 복제, 도전, 아이브]",2646
614388,스타쉽이 드디어 아이브 멤버 각각의 보컬을 살리기 시작한 듯 진짜 딱 본인들한테 맞...,"[본인, 파트, 리즈, 튀어, 듣기, 각각, 음색, 스타, 멤버, 보컬, 아이브]",2645
614267,와 티져듣고 이번에는 평범하게 후크송으로 가려나보다 했는데 진짜 강렬하네 아이브는 ...,"[앨범, 복제, 후크송, 아이브, 티져]",2154
578266,초딩 이야기 많이 하지만 정작 이런 컨셉은 마법소녀에 많이 빠져살던 30대 정도가 ...,"[가족, 마법소녀, 정작, 걸그룹, 초딩, 컨셉, 아이브]",1565


In [85]:
sample_df.shape

(648060, 12)

In [86]:
sample_df.to_csv('keyword_list_after_cleaning.csv')

# 애칭은 나중에..

In [87]:
import re

# 애칭과 본명을 매칭시키는 딕셔너리 정의 (그룹별로 애칭 포함)
nickname_to_name_dict = {
    '뉴진스': {
        '팜하니': '하니', '빵하니': '하니', '뿅오리': '하니', '빠마니': '하니', '아기토끼': '하니',
        '팜토끼': '하니', '파마니': '하니', '팜투리': '하니', '응옥이': '하니', '옥흔이': '하니',
        '팜팜언니': '하니', '팜팜팜': '하니', '감자': '하니', '팜후드': '하니', '팜자후': '하니',
        '코코팜': '하니', '포도알': '하니', '동그라미': '하니', '폼하니': '하니', '팜 미쳤다': '하니',
        '판다팜': '하니', '물개팜': '하니', '하니비': '하니', '팜마리오': '하니', '팜자님': '하니',
        '수달팜': '하니', '팜호초': '하니', '팜다이크': '하니',
        # 다른 뉴진스 멤버 애칭 추가 가능
    },
    # 다른 그룹의 애칭과 본명도 추가 가능
}

# 그룹별 멤버 리스트 정의
group_members = {
    'NewJeans': ['하니', '민지', '다니엘', '혜인', '해린'],
    # 다른 그룹의 멤버도 추가 가능
}

# 댓글에서 애칭을 확인하고 member 열에 본명 추가하는 함수 정의
def add_real_name_to_member_list(row):
    comment = row['comment']
    member_list = row['member'] if isinstance(row['member'], list) else []  # member 열이 없을 경우 빈 리스트로 초기화
    group = row['Group']  # 각 댓글이 속한 그룹

    # 해당 그룹의 멤버만 처리
    if group in nickname_to_name_dict:
        for nickname, real_name in nickname_to_name_dict[group].items():
            # 정규표현식: 단어 경계 (\b) 또는 뒤에 붙는 한국어 조사 인식 (야, 는, 를, 가, 등)
            if re.search(rf'{re.escape(nickname)}([^\w]|$)', comment) and real_name not in member_list:
                # 해당 그룹의 멤버인 경우에만 추가
                if real_name in group_members[group]:
                    member_list.append(real_name)

    return member_list

# sample_df에 member 열이 없으면 새로 생성
if 'member' not in sample_df.columns:
    sample_df['member'] = [[] for _ in range(len(sample_df))]

# sample_df에 대해 apply를 사용하여 member 열 업데이트
sample_df['member'] = sample_df.apply(add_real_name_to_member_list, axis=1)


In [88]:
search_from_comment('레벨', likes=True)

1076


,comment,word_list,likes
309153,뉴진스는 논란이고 뭐고 그냥 행복했으면 좋겠다… 그냥 뉴진스는 어나더레벨임..,"[어나더, 뉴진스, 논란]",3929
330901,그냥 매력이 넘사잖아... 방시혁이 키우는 애덜하고 컨셉이 뭐 비슷하고뭐고간에 애들...,"[뉴진스, 컨셉, 시혁, 방시혁, 매력]",2135
418549,예전부터 느꼈지만 MV는 SM이 할리우드급 아니 미국 MV보다 잘 뽑음.. 이번 노...,"[미국, 지향, 펑키, 예전, 노래, 할리우드, 미래]",987
380216,와 이제야 봤는데 블랙맘바 넥스트레벨 세비지랑 컨셉이 엄청 다른데 엄청 익숙하지않으...,"[블랙맘바, 이제야, 컨셉, 세비지]",858
390541,넥스트 레벨때도 처음에는 먼가 당황스러웠는데 나도 모르게 중독되어 버렸지...이것도...,"[중독, 당황, 스엠, 넥스트 레벨, 넥스트 레벨]",797
371436,2억 조회수 갑시다이야~~~~~~~~~~~ NEXT LEVEL 2억 조회수 닿을때 ...,"[조회수, 쵝오, 수고, 마이, 넥스트 레벨, 넥스트 레벨]",569
391872,넥스트 레벨 이해했다고 노래 난이도 떡상시키네.... 반복 학습하겠습니다.,"[이해, 난이도, 학습, 노래, 반복, 넥스트 레벨, 넥스트 레벨]",561
422848,ses 드컴투 + 소시 갤럭시 슈퍼노바 플라워파워 + 에펙 피노키오 일렉트릭쇼크 +...,"[에스파, 드컴투, 파워, 피노키오, 트릭, 쇼크, 에스파, 정통, 플라워, 소시,...",547
391145,뭔가 느낌이 저번 넥스트 레벨처럼 초반에는 난해하나 나중에는 대박치는 노래일듯..넥...,"[초반, 박치, 나중, 넥레, 킬링, 파트, 노래, 넥스트 레벨, 넥스트 레벨]",449
393613,넥스트 레벨보다 더 에스엠피 스러운 곡이 나왔네요 ㅎㅎ 게다가 뭔가 다들 실력이 더...,"[실력, 에스파, 감탄, 파이팅, 감상, 넥스트 레벨, 넥스트 레벨, 에스엠피]",390


In [89]:
sample_df[sample_df['Group']=='NewJeans']

,Unnamed: 0.1,Group,Title,Unnamed: 0,comment_type,comment_id_key,parent_id_key,comment,author,date,likes,word_list,member
24917,24917,NewJeans,'Hype Boy' Official MV (HANNI ver.),1,node,UgxQhPmB7WZNIjKELJR4AaABAg,NaN,노페이스 언니가 흰색옷 붉은색으로 염색한 눈썹 즉 단월드를 상징하는 겉모습을 하고있...,@won905,2024-09-22T16:24:42Z,1,"[단월드, 흰색, 겉모습, 민희진, 뉴진스, 하이브, 존재, 표현, 은색, 이스, ...",[]
24918,24918,NewJeans,'Hype Boy' Official MV (HANNI ver.),2,reply,NaN,UgxQhPmB7WZNIjKELJR4AaABAg,그리구 하니가 플러스글로벌오디션에 직접 지원한게 노페이스에게 직접 팬이라고 연락한 ...,@won905,2024-09-22T16:29:58Z,1,"[원한, 희진, 플러스, 연락, 직접, 뉴진스, 글로벌, 상징, 이스, 하니, 생존...",[]
24919,24919,NewJeans,'Hype Boy' Official MV (HANNI ver.),3,node,UgyDzAV-30Cy8lXk8J94AaABAg,NaN,인사 씹으면 크닐남...,@jamespark1582,2024-09-21T17:25:20Z,0,"[인사, 크닐남]",[]
24920,24920,NewJeans,'Hype Boy' Official MV (HANNI ver.),5,node,UgzE_gjT223fn9b3JuN4AaABAg,NaN,하니 귀여웡,@jelee2798,2024-09-20T13:10:41Z,0,"[귀여웡, 하니]",[]
24921,24921,NewJeans,'Hype Boy' Official MV (HANNI ver.),8,node,Ugy9TXutM2HSKNwu0vh4AaABAg,NaN,하니 미소가 정말 사람 기분 좋게 한다,@nero_sena,2024-09-17T15:36:11Z,0,"[미소, 하니, 기분]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
547432,46432,NewJeans,Ditto (side A),36651,node,UgyRySjsGtVK8PwPvBB4AaABAg,NaN,1등,@응사마-h6c,2022-12-19T09:00:18Z,1,[],[]
547433,46433,NewJeans,Ditto (side A),36654,node,UgwlpFTp85LoCbwM6Q54AaABAg,NaN,와,@paaaQ,2022-12-19T09:00:16Z,0,[],[]
547434,46434,NewJeans,Ditto (side A),36655,node,UgzOcznxvfrNHRGFGmd4AaABAg,NaN,와,@0x197,2022-12-19T09:00:16Z,1,[],[]
547435,46435,NewJeans,Ditto (side A),36659,node,Ugwewh_EFJDcFEubxDR4AaABAg,NaN,와 뉴깅이들 신곡이다 ㅠㅠㅠ,@ZB1-peach,2022-12-19T09:00:11Z,2,"[뉴깅이들, 신곡]",[]
